# Homework 01 

This Jupyter Notebook file is a template that we provide for the homework.
You _must_ use this template and follow the instructions.
To unify the submission, we require you to store specific intermediate results into _"submission-variables"_.
These values are printed at the _end_ of the jupyter notebook and will be (partially) used to grade your assignment.
Please ensure that these variables have the correct values and aren't overwritten by following code-cells.

In [1]:
# used for submission variables
from copy import deepcopy
import numpy as np
import geopandas
from pathlib import Path

import osmnx

from sentinelsat import SentinelAPI
from datetime import date

from shapely.geometry import Point
import rasterio
import zipfile

from enum import Enum
from typing import List, Sequence
import matplotlib.pyplot as plt

import rasterio.mask

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/ip4rs/share/proj failed


## Sentinel-2 tile download

Download a Sentinel-2 tile from [SciHub Copernicus](https://scihub.copernicus.eu/dhus/#/home) with the following requirements:
- With the sensing date _after_ 01.03.2022
- With the product-type `S2MSI2A`
- _At least_ a cloud cover percentage of 1% but _no more_ than 20%
- Not the tile with the id: 
    - `6e5ad1c9-fc0e-4b93-9b5c-d0d401673206`

_Write_ down (inside of a markdown cell) a city that is part of the downloaded Sentinel-2 tile!

To ensure that no secrets are shared, please use the following code template.
You can create a [TOML](https://en.wikipedia.org/wiki/TOML) file called `.secrets.toml` with the contents:

```toml

scihub_copernicus_user = "<YOUR_PWD>"
scihub_copernicus_pwd = "<YOUR_PWD>"
```

Or you can simply remove your username/password _after_ executing the notebook and _before_ submitting it.

In [2]:
# KEEP
from dynaconf import Dynaconf

settings = Dynaconf(
    envvar_prefix="IP4RS",
    settings_files=["settings.toml", ".secrets.toml"],
)

SENTINEL_API_URL = "https://scihub.copernicus.eu/dhus"
YOUR_USERNAME = "stahir01"
YOUR_PWD = "CitySchool@0"
user = settings.get("scihub_copernicus_user", default=YOUR_USERNAME)
pwd = settings.get("scihub_copernicus_pwd", default=YOUR_PWD)
assert user != r""
assert pwd != r""
###

In [3]:
api = SentinelAPI(user, pwd, SENTINEL_API_URL)

In [4]:
#Since downloading all files takes a lot of time so we take a specific location so we can filter the results
#Select Sanssouci Palace as the location
lat = 52.4042
long = 13.0385

location = geopandas.GeoSeries([Point(long, lat)], crs="EPSG:4326")
location = location.to_wkt()[0]
location

'POINT (13.0385 52.4042)'

In [5]:
# Add here the code to select and download the tile

# The tile-id should be generated here
# for example:
start_date = date(year=2022, month=3, day=2)
end_date = date(year=2022, month=5, day=27)
satellite = "Sentinel-2"
product_type = "S2MSI2A"
tile_id = "6e5ad1c9-fc0e-4b93-9b5c-d0d401673206"


tile_results = api.query(location,
                         date=(start_date,end_date), 
                         platformname=satellite, 
                         producttype=product_type,
                         cloudcoverpercentage=(1,20))



#api.download_all(tile_results)
products_df = api.to_geodataframe(tile_results)
assert not products_df.empty
products_df



,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid,geometry
365e1758-960c-42ef-a284-1c0b739a28d2,S2A_MSIL2A_20220518T100601_N0400_R022_T33UUU_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-18T10:06:01.024Z, Instrument: MS...",false,2022-05-18 16:28:16,2022-05-18 10:06:01.024,2022-05-18 10:06:01.024,2022-05-18 18:24:30.654,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_ATOS_20220518T162816_S2022...,S2A_OPER_MSI_L2A_TL_ATOS_20220518T162816_A0360...,S2A_MSIL2A_20220518T100601_N0400_R022_T33UUU_2...,365e1758-960c-42ef-a284-1c0b739a28d2,"MULTIPOLYGON (((12.36233 52.23145, 13.67854 52..."
c9fab88d-ac41-4882-baf6-9333982cca29,S2A_MSIL2A_20220518T100601_N0400_R022_T32UQD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-18T10:06:01.024Z, Instrument: MS...",false,2022-05-18 16:28:16,2022-05-18 10:06:01.024,2022-05-18 10:06:01.024,2022-05-18 18:19:07.541,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_ATOS_20220518T162816_S2022...,S2A_OPER_MSI_L2A_TL_ATOS_20220518T162816_A0360...,S2A_MSIL2A_20220518T100601_N0400_R022_T32UQD_2...,c9fab88d-ac41-4882-baf6-9333982cca29,"MULTIPOLYGON (((13.53103 52.17548, 13.63418 53..."
aa06fd73-3af8-4a35-bc1e-63f50e5fc106,S2A_MSIL2A_20220508T100601_N0400_R022_T32UQD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-08T10:06:01.024Z, Instrument: MS...",false,2022-05-08 17:38:19,2022-05-08 10:06:01.024,2022-05-08 10:06:01.024,2022-05-08 19:36:13.052,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_ATOS_20220508T173819_S2022...,S2A_OPER_MSI_L2A_TL_ATOS_20220508T173819_A0359...,S2A_MSIL2A_20220508T100601_N0400_R022_T32UQD_2...,aa06fd73-3af8-4a35-bc1e-63f50e5fc106,"MULTIPOLYGON (((13.53103 52.17548, 13.63418 53..."
a19251bd-6e0a-4ae5-a1d1-749ff966adea,S2A_MSIL2A_20220508T100601_N0400_R022_T33UUU_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-08T10:06:01.024Z, Instrument: MS...",false,2022-05-08 17:38:19,2022-05-08 10:06:01.024,2022-05-08 10:06:01.024,2022-05-08 19:29:22.527,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_ATOS_20220508T173819_S2022...,S2A_OPER_MSI_L2A_TL_ATOS_20220508T173819_A0359...,S2A_MSIL2A_20220508T100601_N0400_R022_T33UUU_2...,a19251bd-6e0a-4ae5-a1d1-749ff966adea,"MULTIPOLYGON (((12.36470 52.23149, 13.67854 52..."
53f211a7-42d5-4a67-8fd6-69d70308a85a,S2B_MSIL2A_20220423T100549_N0400_R022_T33UUU_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-04-23T10:05:49.024Z, Instrument: MS...",false,2022-04-23 13:43:29,2022-04-23 10:05:49.024,2022-04-23 10:05:49.024,2022-04-23 17:46:09.092,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS2_20220423T134329_S2022...,S2B_OPER_MSI_L2A_TL_VGS2_20220423T134329_A0267...,S2B_MSIL2A_20220423T100549_N0400_R022_T33UUU_2...,53f211a7-42d5-4a67-8fd6-69d70308a85a,"MULTIPOLYGON (((12.35500 52.23131, 13.67854 52..."
e681985a-dc3a-40fc-8406-2f122913dc45,S2B_MSIL2A_20220423T100549_N0400_R022_T32UQD_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-04-23T10:05:49.024Z, Instrument: MS...",false,2022-04-23 13:43:29,2022-04-23 10:05:49.

In [10]:

#We select the following tile
products_df = products_df.head(1)
products_df
products_df[["title","summary", "geometry"]].explore()
#tile_id = products_df.iloc[0] #To check details regarding the tile we selected
#tile_id



In [ ]:
#Now we download the datafame of the tile


In [ ]:
### DO NOT EDIT
submission_tile_id = deepcopy(tile_id)
##

## Valid sub-region

From the previously downloaded tile, extract a _valid_ subregion.
Convert the subregion to a `uint8` array without dropping important information (i.e., not simply calling `np.uint8` on the input).

In [ ]:
### Add here the code to select a valid subregion

subregion_uint8 = ...

In [ ]:
### DO NOT EDIT
submission_subregion_uint8 = deepcopy(subregion_uint8)
###

### Visualize the valid sub-region

Visualize a _single band_ from the valid subregion _without_ applying any explicit or implicit normalization strategy.
The input array passed to `plt.imshow` has to be in the range `[0, 255]`. This range should also be reflected in the `vmin/vmax` parameters.

In [ ]:
# Add the code to visualize the single band
# without normalization here:

# imshow_inp is the array that should be passed to plt.imshow
imshow_inp = ...

In [ ]:
# DO NOT EDIT
submission_imshow_inp = deepcopy(imshow_inp)
###

## Morphological operators & Thresholding

The goal of this section is to combine morphological operators with thresholding strategies to create a binary/output mask that can then be used to segment the region/object of interest.
You are _not_ required to perfectly segment the object of interest. It is sufficient if the visual results _indicate_ the object of interest.

You have to meet the following requirements:
- You are _not_ allowed to use simple rectangular structuring elements.
- The values of the output mask must either be 0 or 1
    - 0 is used to indicate the absence of the object
    - 1 is used to indicate the presence of the object

In [ ]:
# Add code here to apply morphological operators
# and thresholding strategy
# You are allowed to change/mix the order

output_mask = ...

In [ ]:
# DO NOT EDIT
submission_output_mask = deepcopy(output_mask)
###

To visually evaluate your result, use the output mask as an overlay for the RGB channels of the input array.
The colored output should be normalized to enhance the viewing experiment.
You can choose any _valid_ normalization strategy/method for the RGB channels.

In [ ]:
# Visualize the overlayed image

### Discussion

Discuss what you have learned when applying the morphological operators and thresholding strategies.
What didn't work? What issues did you have? What helped?
How did you decide to use the variables you have chosen?

It is not necessary to give an in-depth discussion about each value.
It should be clear, however, that you have tried different configurations and why you have decided on the current configuration.

Here you should write discussion:

## Submission

- Follow the instructions mentioned in this notebook.
- You are _not_ allowed to install other libraries.
    - The submitted jupyter notebook should be able to run as-is on paperspace and/or the provided Docker image.
- Explain the steps that went into the decision making of the specific values.
- It is essential that each homework should be done individually!
- The notebook should have been executed once before submitting it
    - The notebook should _not_ be _cleaned_ before submitting

Each student should submit the following materials to the ISIS system:
- One `zip` file containing the jupyter notebook named as: `IP4RS2022-HW1_<NAME_SURNAME>.zip`



### Submission variables

Please ensure that these variables are correct after you have restarted your notebook and executed all cells from top to bottom.
You are _not allowed_ to modify the following cells:

In [ ]:
import rich

rich.print(f"{submission_tile_id=}")
rich.print(f"{submission_subregion_uint8=}")
rich.print(f"{submission_output_mask=}")